## Engenharia de Machine Learning

### Template - Predição climática

In [1]:
# !pip install awswrangler

In [2]:
import os
import sys
import json
import time
import boto3
import numpy as np
import pandas as pd
from io import StringIO
import awswrangler as wr
from datetime import datetime
import matplotlib.pyplot as plt

import sagemaker
from sagemaker.model import Model
from sagemaker.session import Session
from sagemaker.model_monitor import DefaultModelMonitor
from sagemaker.model_monitor.dataset_format import DatasetFormat

sys.path.append(os.path.dirname(os.getcwd())+'/src')
import utils

In [3]:
session = boto3.Session()
sm = session.client('sagemaker')
role = sagemaker.get_execution_role()
account_id = session.client('sts').get_caller_identity()['Account']
print(sm, '\n', role, '\n', account_id)

 arn:aws:iam::478704051461:role/service-role/AmazonSageMaker-ExecutionRole-20220317T121940 
 478704051461


---

### Treinamento e implantação do modelo

In [4]:
# bucket e endereços dos datasets de treino e validação
bucket='ons-ds-mlops'
training_path = 'framework-overview/data/processed/train/dataset.csv'
validation_path = 'framework-overview/data/processed/validation/dataset.csv'
trainset_uri=f's3://{bucket}/{training_path}'
validationset_uri=f's3://{bucket}/{validation_path}'

# container contendo o modelo
container = sagemaker.image_uris.retrieve(region=session.region_name, framework='xgboost', version='latest')

# definição de hiperparametros
hyperparameters={
    'eta':0.4,
    'gamma':1,
    'max_depth':3,
    'num_round':20,
    'min_child_weight':8,
    'subsample':0.6,
    'objective':"multi:softmax",
    'num_class':5
}

# uri para artefatos do modelo
model_uri = f's3://{bucket}/framework-overview/artifacts/model'

# treinar o modelo
model = utils.train_model(
    container=container,
    trainset_uri=trainset_uri,
    validationset_uri=validationset_uri,
    model_uri=model_uri,
    hyperparameters=hyperparameters
    )

2022-08-07 23:42:11 Starting - Starting the training job...
2022-08-07 23:42:34 Starting - Preparing the instances for trainingProfilerReport-1659915731: InProgress
......
2022-08-07 23:43:37 Downloading - Downloading input data...
2022-08-07 23:43:57 Training - Downloading the training image......
2022-08-07 23:45:08 Training - Training image download completed. Training in progress..Arguments: train
[2022-08-07:23:45:11:INFO] Running distributed xgboost training.
[2022-08-07:23:45:11:INFO] Number of hosts: 2, master IP address: 10.0.225.248, host IP address: 10.0.214.210.
[2022-08-07:23:45:11:INFO] Finished Yarn configuration files setup.
starting datanode, logging to /opt/amazon/hadoop/logs/hadoop--datanode-ip-10-0-214-210.ec2.internal.out
Arguments: train
[2022-08-07:23:45:11:INFO] Running distributed xgboost training.
[2022-08-07:23:45:11:INFO] Number of hosts: 2, master IP address: 10.0.225.248, host IP address: 10.0.225.248.
[2022-08-07:23:45:11:INFO] Finished Yarn configuration

In [5]:
# listar arquivos de modelos no bucket
s3_client = session.client('s3')
result = s3_client.list_objects(Bucket=bucket, Prefix='framework-overview/artifacts/model')
files = [file.get('Key') for file in result.get('Contents')]
files = [f's3://{bucket}/{file}' for file in files if 'model.tar.gz' in file]

In [6]:
model_path = files[-1]
model_path

's3://ons-ds-mlops/framework-overview/artifacts/model/xgboost-2022-08-07-23-42-11-285/output/model.tar.gz'

In [7]:
# configuração de captura de dados
data_capture_uri = f's3://{bucket}/framework-overview/data/capture'
data_capture_config = utils.get_data_capture_config(
    storage_uri=data_capture_uri,
    sampling_percentage=100
    )

In [8]:
# implantação do modelo em um endpoint
stage = 'dev'
timehash = datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
endpoint_name = f"predicao-climatica-{stage}-{timehash}"
print(f'nome do endpoint: {endpoint_name}')

endpoint = utils.deploy_model(
    container=container,
    model_uri=model_path,
    endpoint_name=endpoint_name,
    data_capture_config=data_capture_config
)

nome do endpoint: predicao-climatica-dev-2022-08-07-23-47-30
-----------------------!Endpoint predicao-climatica-dev-2022-08-07-23-47-30 criado com sucesso


---

### Provisionamento do monitor de modelos

In [9]:
# declaração do baseline
baseline_path = f'framework-overview/model-monitor/baseline'

model_monitor = utils.create_model_monitor_baseline(
    bucket=bucket,
    baseline_path=baseline_path,
    trainset_uri=trainset_uri
    )
model_monitor

baseline_data_uri: s3://ons-ds-mlops/framework-overview/model-monitor/baseline/data
baseline_results_uri: s3://ons-ds-mlops/framework-overview/model-monitor/baseline/results

Job Name:  baseline-suggestion-job-2022-08-07-23-59-04-210
Inputs:  [{'InputName': 'baseline_dataset_input', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://ons-ds-mlops/framework-overview/data/processed/train/dataset.csv', 'LocalPath': '/opt/ml/processing/input/baseline_dataset_input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'monitoring_output', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://ons-ds-mlops/framework-overview/model-monitor/baseline/results', 'LocalPath': '/opt/ml/processing/output', 'S3UploadMode': 'EndOfJob'}}]
..........................2022-08-08 00:03:18,118 - matplotlib.font_manager - INFO - Generating new fontManager, this may take some time...
2022-08-08 00:03:18.642509: W tensorflo

In [10]:
# verificar criação do baseline
report_files = utils.list_model_monitor_report_files(
    bucket=bucket,
    baseline_path=baseline_path
    )
report_files

['framework-overview/model-monitor/baseline/results/constraints.json',
 'framework-overview/model-monitor/baseline/results/statistics.json']

In [11]:
# verificar schema
schema_df = utils.get_model_monitor_schema(
    model_monitor
    )
schema_df.head()

/root/ons-mlops-overview/src/utils.py:386: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  schema_df = pd.io.json.json_normalize(baseline_job.baseline_statistics().body_dict['features'])


,name,inferred_type,numerical_statistics.common.num_present,numerical_statistics.common.num_missing,numerical_statistics.mean,numerical_statistics.sum,numerical_statistics.std_dev,numerical_statistics.min,numerical_statistics.max,numerical_statistics.distribution.kll.buckets,numerical_statistics.distribution.kll.sketch.parameters.c,numerical_statistics.distribution.kll.sketch.parameters.k,numerical_statistics.distribution.kll.sketch.data
0,_c0,Integral,1096,0,2.416058,2648.00,1.471219,0.0,4.0,"[{'lower_bound': 0.0, 'upper_bound': 0.4, 'cou...",0.64,2048.0,"[[2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 4.0, 1.0,..."
1,_c1,Fractional,1096,0,2.998905,3286.80,6.310361,0.0,54.1,"[{'lower_bound': 0.0, 'upper_bound': 5.41, 'co...",0.64,2048.0,"[[0.0, 10.9, 0.8, 20.3, 1.3, 2.5, 0.0, 0.0, 4...."
2,_c2,Fractional,1096,0,3.268248,3582.00,1.470958,0.4,9.5,"[{'lower_bound': 0.4, 'upper_bound': 1.31, 'co...",0.64,2048.0,"[[4.7, 4.5, 2.3, 4.7, 6.1, 2.2, 2.3, 2.0, 3.4,..."
3,_c3,Fractional,1096,0,8.075091,8850.30,3.795535,0.6,18.9,"[{'lower_bound': 0.5999999999999996, 'upper_bo...",0.64,2048.0,"[[7.800000000000001, 7.8, 4.499999999999999, 6..."
4,_c4,Fractional,1096,0,12.072217,13231.15,6.010556,-3.8,26.7,"[{'lower_bound': -3.8, 'upper_bound': -0.74999...",0.64,2048.0,"[[8.9, 6.699999999999999, 9.45, 8.899999999999..."


In [12]:
# verificar constraints
constraints_df = utils.get_model_monitor_constraints(
    model_monitor
    )
constraints_df.head()

/root/ons-mlops-overview/src/utils.py:405: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  constraints_df = pd.io.json.json_normalize(baseline_job.suggested_constraints().body_dict['features'])


,name,inferred_type,completeness,num_constraints.is_non_negative
0,_c0,Integral,1.0,True
1,_c1,Fractional,1.0,True
2,_c2,Fractional,1.0,True
3,_c3,Fractional,1.0,True
4,_c4,Fractional,1.0,False


In [13]:
# configurar rotina de monitoramento
schedule_name = f'schedule-mm-predicao-climatica-{timehash}'
output_monitor_uri = f's3://{bucket}/framework-overview/model-monitor/output'
model_monitor = utils.create_monitoring_schedule(
    model_monitor,
    endpoint_name,
    monitor_schedule_name=schedule_name,
    output_s3_uri=output_monitor_uri
    )

---

### Verificar análises do monitor

Após o provisionamento do endpoint e do sistema de monitoramento de modelos (data quality), uma massa de dados foi usada para produzir uma análise. Os resultados podem ser acessados através dos seguintes comandos:

In [15]:
# listar execuções do monitor - segundo cronograma programado
model_monitor.list_executions()

In [16]:
# descrição do trabalho de monitoramento
model_monitor.describe_schedule()

{'MonitoringScheduleArn': 'arn:aws:sagemaker:us-east-1:478704051461:monitoring-schedule/schedule-mm-predicao-climatica-2022-08-01-00-08-22',
 'MonitoringScheduleName': 'schedule-mm-predicao-climatica-2022-08-01-00-08-22',
 'MonitoringScheduleStatus': 'Scheduled',
 'MonitoringType': 'DataQuality',
 'CreationTime': datetime.datetime(2022, 8, 1, 0, 24, 12, 198000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2022, 8, 1, 1, 13, 51, 358000, tzinfo=tzlocal()),
 'MonitoringScheduleConfig': {'ScheduleConfig': {'ScheduleExpression': 'cron(0 * ? * * *)'},
  'MonitoringJobDefinitionName': 'data-quality-job-definition-2022-08-01-00-24-11-918',
  'MonitoringType': 'DataQuality'},
 'EndpointName': 'predicao-climatica-dev-2022-08-01-00-08-22',
 'LastMonitoringExecutionSummary': {'MonitoringScheduleName': 'schedule-mm-predicao-climatica-2022-08-01-00-08-22',
  'ScheduledTime': datetime.datetime(2022, 8, 1, 1, 0, tzinfo=tzlocal()),
  'CreationTime': datetime.datetime(2022, 8, 1, 1, 6, 7, 3

In [21]:
# identificar status do trabalho - com violações ou sem violações
model_monitor.describe_schedule()['LastMonitoringExecutionSummary']['MonitoringExecutionStatus']

'CompletedWithViolations'

A massa de dados foi propositalmente alterada em relação aos dados de referência (baseline) para promover o retorno de violações. Este retorno pode ser usado na criação de uma notificação e no acionamento de um pipeline de retreino.

---

### Limpeza do ambiente

In [15]:
# Deletar rotina de monitoramento
model_monitor.delete_monitoring_schedule()


Deleting Monitoring Schedule with name: schedule-mm-predicao-climatica-2022-08-07-23-47-30


In [16]:
# Deletar endpoint
sm.delete_endpoint(EndpointName=endpoint_name)

{'ResponseMetadata': {'RequestId': 'c7366723-4c80-453f-80a2-ef1a6e3e3741',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'c7366723-4c80-453f-80a2-ef1a6e3e3741',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Mon, 08 Aug 2022 00:43:47 GMT'},
  'RetryAttempts': 0}}